# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [1]:
# dependencies
!pip install --upgrade boto3
!pip install langchain
!pip install langchainhub
!pip install anthropic

## check bedrock availability

In [2]:
def usage_demo():
    """
    Shows how to list the available foundation models.
    This demonstration gets the list of available foundation models and
    prints their respective summaries.
    """
    logging.basicConfig(level=logging.INFO)
    print("-" * 88)
    print("Welcome to the Amazon Bedrock demo.")
    print("-" * 88)

    bedrock_client = boto3.client(service_name="bedrock", region_name="us-east-1")

    wrapper = BedrockWrapper(bedrock_client)

    print("Listing the available foundation models.")

    try:
        for model in wrapper.list_foundation_models():
            print_model_details(model)
    except ClientError:
        logger.exception("Couldn't list foundation models.")
        raise

    print("Getting the details of an individual foundation model.")

    model_id = "amazon.titan-embed-text-v1"

    try:
        print_model_details(wrapper.get_foundation_model(model_id))
    except ClientError:
        logger.exception(f"Couldn't get foundation model {model_id}.")
        raise

In [4]:
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock
from utils.BedrockWrapper import BedrockWrapper
from utils.GamePlayer import GamePlayer
from utils.GameAssistant import GameAssistant
from utils.GameMaster import GameMaster
from utils.PeTemplates import *
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error
from utils import print_model_details
        
from botocore.exceptions import ClientError

usage_demo()
#boto3_bedrock = bedrock.get_bedrock_client()

#listModels = bedrock.list_foundation_models(byProvider='meta')

# print_ww("hello")
# Info("hello")

# GA = GameAssistant(template_assistant_role, 1000)
# print(GA.agent.prompt.template)

03/05/2024 02:32:45 AM Got 45 foundation models.


----------------------------------------------------------------------------------------
Welcome to the Amazon Bedrock demo.
----------------------------------------------------------------------------------------
Listing the available foundation models.

 Model: amazon.titan-tg1-large
------------------------------------------
 Name: Titan Text Large
 Provider: Amazon
 Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large
 Lifecycle status: ACTIVE
 Input modalities: ['TEXT']
 Output modalities: ['TEXT']
 Supported customizations: []
 Supported inference types: ['ON_DEMAND']
 Response streaming supported: True

 Model: amazon.titan-image-generator-v1:0
------------------------------------------
 Name: Titan Image Generator G1
 Provider: Amazon
 Model ARN: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0
 Lifecycle status: ACTIVE
 Input modalities: ['TEXT', 'IMAGE']
 Output modalities: ['IMAGE']
 Supported customizations: ['FINE_TUNIN

## Test Part

In [ ]:
from collections import Counter

def find_most_frequent(arr):
  freq = Counter(arr)
  max_count = max(freq.values())
  
  return [k for k, v in freq.items() if v == max_count]

vote_names = ['p1','p1','p2','p2','p3','p5','p4','p4']
AA = Counter(vote_names)
print(AA.most_common())

print(find_most_frequent(vote_names))

aa = []

if len(aa) != 1:
    print(1)

In [5]:
from utils.PeTemplates import *
from langchain.agents import tool
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field
    
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

print(prompt)

model = claude3_Sonnet
# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'
 Here is a joke formatted to the given JSON schema:

```json
{"setup": "Why can't a bicycle stand on its own?", "punchline": "It's too tired!"}
```

Joke(setup="Why can't a bicycle stand on its own?", punchline="It's too tired!")

In [ ]:
from utils.CustomTools import *
from utils.PeTemplates import *
from langchain.agents import tool

# - 夜晚投票(狼人专属行动): WolfVote 参数: target=存活玩家
# - 夜晚查验(预言家专属行动): ProphetCheck 参数: target=存活玩家
# - 白天怀疑(所有玩家白天可选行动, 非投票): PlayerDoubt 参数: target=存活玩家 
# - 白天投票: PlayerVote 参数: target=存活玩家 
# - 白天讨论: Debate 参数: content=思考/理由 
# - 玩家信息: GetAllPlayersName 参数: 无 
# - 死亡遗言: DeathWords 参数: content=给予玩家线索
# - 玩家弃权: Pass 参数: 无 
# - 其他动作: Pass 参数: 无 

@tool
def WolfVote(target: str) -> str:
    """夜晚投票(狼人专属行动)"""
    return "WolfVote"

@tool
def ProphetCheck(target: str) -> str:
    """夜晚查验(预言家专属行动)"""
    return "ProphetCheck"

@tool
def PlayerDoubt(target: str) -> str:
    """白天怀疑(所有玩家白天可选行动, 非投票)"""
    return "PlayerDoubt"

@tool
def PlayerVote(target: str) -> str:
    """白天投票"""
    return "PlayerVote"

@tool
def Debate(target: str) -> str:
    """白天讨论"""
    return "Debate"

@tool
def GetAllPlayersName(target: str) -> str:
    """玩家信息"""
    return "GetAllPlayersName"

@tool
def DeathWords(content: str) -> str:
    """死亡遗言"""
    return "DeathWords"

@tool
def Pass(content: str) -> str:
    """玩家弃权,说明理由"""
    return "Pass"

 
# print(f"search.name:{search.name}")
# print(f"search.description:{search.description}")
# print(f"search.args:{search.args}")

# Define a list of tools
tools = [
    Tool(
        name = "WolfVote",
        func=WolfVote.run,
        description="夜晚投票(狼人专属行动)"
    ),
    Tool(
        name = "ProphetCheck",
        func=ProphetCheck.run,
        description="夜晚查验(预言家专属行动)"
    ),
    Tool(
        name = "PlayerDoubt",
        func=PlayerDoubt.run,
        description="白天怀疑(所有玩家白天可选行动, 非投票)"
    ),
    Tool(
        name = "PlayerVote",
        func=PlayerVote.run,
        description="白天投票"
    ),
    Tool(
        name = "Pass",
        func=Pass.run,
        description="玩家弃权,说明理由"
    )
]

# Using tools, the LLM chain and output_parser to make an agent
tool_names = [tool.name for tool in tools]

prompt = CustomPromptTemplate(
    template=template_werewolf_role,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "history", 'agent_scratchpad', 'tools', 'tool_names']
)
#print(prompt)
# output_parser = CustomOutputParser()

llm = claude_instant_llm

# LLM chain consisting of the LLM and a prompt
# llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = create_structured_chat_agent(llm, tools, prompt)

# agent = LLMSingleActionAgent(
#     llm_chain=llm_chain, 
#     output_parser=output_parser,
#     # We use "Observation" as our stop sequence so it will stop when it receives Tool output
#     # If you change your prompt template you'll need to adjust this as well
#     stop=["\nObservation:"], 
#     allowed_tools=tool_names
# )

# Initiate the agent that will respond to our queries
# Set verbose=True to share the CoT reasoning the LLM goes through
memory = ConversationBufferWindowMemory(k=2)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=False)

agent_executor.invoke({"input":"第一个晚上，你支持的玩家该如何行动？"})


In [8]:
from utils.GamePlayer import GamePlayer
from utils.GameMaster import GameMaster
from utils.PeTemplates import *
from utils import ParseJson, print_ww, Print, Info, Debug, Warn, Error

llm = claude3_Sonnet
GM = GameMaster(10, llm, False)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

03/05/2024 02:50:59 AM 	===== P1:存活,P2:存活,P3:淘汰,P4:存活,P5:淘汰,P6:存活,P7:存活,P8:存活 ResetGame =====
03/05/2024 02:50:59 AM P1 is 预言家
03/05/2024 02:50:59 AM P2 is 村民
03/05/2024 02:50:59 AM P3 is 狼人
03/05/2024 02:50:59 AM P4 is 村民
03/05/2024 02:50:59 AM P5 is 村民
03/05/2024 02:50:59 AM P6 is 狼人
03/05/2024 02:50:59 AM P7 is 村民
03/05/2024 02:50:59 AM P8 is 村民
03/05/2024 02:50:59 AM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 RunGame =====
03/05/2024 02:50:59 AM 	===== input_tokens: 0 output_tokens 0 ======
03/05/2024 02:50:59 AM 	===== elapsed_time: 0.007664203643798828 ======
03/05/2024 02:50:59 AM 	===== NIGHT 1 PreAction ======
03/05/2024 02:50:59 AM 	===== NIGHT 1 DoAction ======
03/05/2024 02:50:59 AM 		===== DoMemory NIGHT 1 P1 ======
03/05/2024 02:50:59 AM 		===== DoReflect NIGHT 1 P1 ======
03/05/2024 02:50:59 AM 		===== DoAnswer NIGHT 1 P1 ======
03/05/2024 02:50:59 AM 	QUESTION: 现在是NIGHT 1,你支持的玩家是P1(预言家身份)..目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第1天

 {"action":"ProphetCheck","target":"P3"}

思考:
根据游戏规则,第一个夜晚死亡的一定是村民或者预言家,而不是狼人。
昨天白天,P3的表现让我感觉他在混淆是非,很有可能在帮助狼人分散火力。
作为预言家,第一个夜晚最好查验他,以了解其身份,为后续游戏提供线索。

03/05/2024 02:51:02 AM 		===== DoAction NIGHT 1 P1 ======
03/05/2024 02:51:02 AM [玩家P1于时间NIGHT 1, 执行动作为:{'action': 'ProphetCheck', 'target': 'P3'}]P3:狼人
03/05/2024 02:51:02 AM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P1', 'status': 1, 'role': '预言家', 'character': '独立思考', 'response': ['{"action":"ProphetCheck","target":"P3"}']}

03/05/2024 02:51:02 AM 		===== DoMemory NIGHT 1 P3 ======
03/05/2024 02:51:02 AM 		===== DoReflect NIGHT 1 P3 ======
03/05/2024 02:51:02 AM 		===== DoAnswer NIGHT 1 P3 ======
03/05/2024 02:51:02 AM 	QUESTION: 现在是NIGHT 1,你支持的玩家是P3(狼人身份,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第1天夜晚，该如何行动?


 {"action":"WolfVote","target":"P5"}

思考:
根据游戏规则,第一个夜晚死亡的一定是村民或者预言家,狼人没必要第一晚上自杀。P5无任何嫌疑信息暴露,选择他可以引起最大混乱,为后续游戏制造有利条件。

03/05/2024 02:51:05 AM 		===== DoAction NIGHT 1 P3 ======
03/05/2024 02:51:05 AM 
 [ROUND ACTION]={'time': 'NIGHT 1', 'player': 'P3', 'status': 1, 'role': '狼人', 'character': '组织者', 'response': ['{"action":"WolfVote","target":"P5"}']}

03/05/2024 02:51:05 AM 		===== DoMemory NIGHT 1 P6 ======


...

03/05/2024 02:54:28 AM 		===== DoAnswer NIGHT 2 P6 ======
03/05/2024 02:54:28 AM 	QUESTION: 现在是NIGHT 2,你支持的玩家是P6(狼人身份,本阵营为:P3,P6).目前场上玩家:P1:存活,P2:存活,P3:存活,P4:淘汰,P5:淘汰,P6:存活,P7:存活,P8:存活(逗号为分割符).现在是第2天夜晚，该如何行动?


 {"action":"WolfVote","target":"P7"}

理由是:P7表现低调,没有被质疑,是当前最安全的目标。除掉可以引导舆论,为下一步埋线索。

03/05/2024 02:54:30 AM 		===== DoAction NIGHT 2 P6 ======
03/05/2024 02:54:30 AM 
 [ROUND ACTION]={'time': 'NIGHT 2', 'player': 'P6', 'status': 1, 'role': '狼人', 'character': '过激型', 'response': ['{"action":"WolfVote","target":"P7"}']}

03/05/2024 02:54:30 AM 	===== NIGHT 2 PostAction ======
03/05/2024 02:54:30 AM 	 [wolf_votes]: ['P7'], [player_vote_name]: Counter({'P7': 2})
03/05/2024 02:54:30 AM 
 [WOLF VOTE]={'time': 'NIGHT 2', 'player': 'P7', 'status': 0, 'role': '村民', 'character': '观察家', 'response': '玩家P7于NIGHT 2被狼人投票而出局'}



 第1晚,预言家P1查验P3身份,P3、P6两名狼人同意投票杀死P5。白天,P5死亡遗言指出P3可疑。P3否认并质疑P5,P4支持P5,P6、P8提醒重视线索而非个人判断。大家讨论后,P1、P3、P6三人同意投票P4,P4遭淘汰。P4死亡遗言再次提醒重视P3举动。

第2晚,预言家P1查验P2身份。P3、P6再次同意投票杀死P7。白天,狼人成功消灭所有村民,狼人胜利。

03/05/2024 02:54:36 AM 狼人胜利!
03/05/2024 02:54:36 AM 游戏结束.
03/05/2024 02:54:36 AM ===== P1:存活,P2:存活,P3:存活,P4:淘汰,P5:淘汰,P6:存活,P7:淘汰,P8:存活 EndGame =====
03/05/2024 02:54:36 AM 	===== input_tokens: 3079652 output_tokens 6971 ======
03/05/2024 02:54:36 AM 	===== elapsed_time: 216.95154476165771 ======
